# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.5.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
#Image augmentation - implement this code to avoid overfitting. It is basically a geometric transformation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#"rescale" is basically feature scaling. we are dividing the values in each pixel by 255 to normalize the values to be within 0,1
#"shear range" , "zoom range", and horizontal flips are the geometric transformations to avoid overfitting. 
training_set = train_datagen.flow_from_directory('/Users/sophiaty/Desktop/SRA Codes/Imaging_Dataset/Train_Set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

#targetsize refers to the final size of the image
#batchsize refers to the number of items in a batch to be processed by CNN. We can do 32 bcause we have thousands of images
#class_mode is binary because it is either cat or dog! You need to change this for more option

Found 26 images belonging to 5 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
#you only perform the feature scaling by the same value (dividing by 255)
#you should not transform your test data because this is a "Data leak"
test_set = test_datagen.flow_from_directory('/Users/sophiaty/Desktop/SRA Codes/Imaging_Dataset/Test_Set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 13 images belonging to 5 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential() #this initializes the CNN to be a sequential class not a graph one


### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',input_shape=[64,64,3]))
#this adds a 2D conv layer.
#filter? arbitrary number that was chosen, you can try different number of filters
#kernel_size = refers to the dimensions of the filter (3 means 3x3 square))
#activation =refers to the type of activation function
#input_shape contains 64,64,3 to indicate that the images have been rescaled to be 64x64 that is colored (RGB =3).
#input_shape's last arg would =1 if the image is black and white

### Step 2 - Pooling

In [7]:
#applies max pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#pool-size arg refers to a 2x2 box that is used to pool information from the previous layer
#strides refers to the shift size, in this case two boxes/pixels 

### Adding a second convolutional layer

In [8]:
#add second conv layer with max pooling. 
#input_Shape no longer necessary. We only want that in the first layer to attach CNN to the input images
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


### Step 3 - Flattening

In [9]:
#create a vector that will act as the input to the regular ANN aka full connection layer
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
#this adds a fullly connected layer. There are more units or neurons because we have a more complex system. 
#we are continuing to use the rectifier function as the activation function
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))
#this adds an output layer that has 5 neurons with a softmax activation function

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
#this step connects the CNN to optimizer, loss function, and metrics.
cnn.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=[tf.keras.metrics.SpecificityAtSensitivity(0.2)])
#optimizer adam?
#loss function is a categorical crossentropy
#metrics: accuracy of prediction is calculated but we also want to assess specficity


### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 80)
#this fits our model to the training set. We are using the test_Set as the validation test, epochs refer to total number of runs


Epoch 1/80
1/1 [==============================] - 2s 2s/step - loss: 1.5691 - specificity_at_sensitivity: 0.9327 - val_loss: 1.5874 - val_specificity_at_sensitivity: 0.8846
Epoch 2/80
1/1 [==============================] - 1s 512ms/step - loss: 1.3184 - specificity_at_sensitivity: 0.9808 - val_loss: 1.5458 - val_specificity_at_sensitivity: 0.9231
Epoch 3/80
1/1 [==============================] - 1s 567ms/step - loss: 1.2964 - specificity_at_sensitivity: 0.9231 - val_loss: 1.4743 - val_specificity_at_sensitivity: 0.9038
Epoch 4/80
1/1 [==============================] - 0s 489ms/step - loss: 1.2164 - specificity_at_sensitivity: 0.9615 - val_loss: 1.4372 - val_specificity_at_sensitivity: 0.9423
Epoch 5/80
1/1 [==============================] - 1s 532ms/step - loss: 1.2065 - specificity_at_sensitivity: 0.9808 - val_loss: 1.4455 - val_specificity_at_sensitivity: 0.9038
Epoch 6/80
1/1 [==============================] - 1s 571ms/step - loss: 1.1875 - specificity_at_sensitivity: 0.9808 - val_l